In [1]:
DIR="/vast/scratch/users/mueller.h/hackathon"
#source("/Users/harrisonm/data/WEHI/Conferences/OZSC_Hackathon/scripts/st_python/r/utils.R")
#source("/Users/harrisonm/data/WEHI/Conferences/OZSC_Hackathon/scripts/st_python/r/transfer.R")
OUTDIR = file.path(DIR, "scripts/hackathon_multiomic_embeddings/results")
# Load libraries

In [27]:
library(Seurat)
library(ggplot2)
library(edgeR)
library(dplyr)


In [89]:
atac <- readRDS(file.path(DIR, "data", "qc", "Annotated_processed_unpaired_ATAC.RDS"))
rna <- readRDS(file.path(DIR, "data", "qc", "processed_Annotated_unpaired_RNA.RDS"))

### get embeddings

In [4]:
rna <- RunPCA(rna, npcs = 20)
rna <- RunUMAP(rna, dims = 1:20)
rna <- RunTSNE(rna, dims = 1:20)

PC_ 1 
Positive:  IL32, CD247, TRAC, CAMK4, LTB, IL7R, BACH2, LEF1, SYNE2, INPP4B 
	   BCL2, TRBC2, THEMIS, RORA, ANK3, TXK, CD69, TRBC1, MLLT3, TMSB4X 
	   RASGRF2, LINC01934, NELL2, TAFA1, ACTG1, PATJ, MAL, TMSB10, PCAT1, GZMM 
Negative:  PLXDC2, SLC8A1, FCN1, LRMDA, TYMP, JAK2, CYBB, MCTP1, DMXL2, NAMPT 
	   ZEB2, IRAK3, TBXAS1, RBM47, GAB2, LYN, LYZ, TLR2, VCAN, HCK 
	   LRRK2, MNDA, SAT1, LYST, CLEC7A, TNFAIP2, CSF3R, CD36, MARCH1, DENND1A 
PC_ 2 
Positive:  NIBAN3, IGHM, BANK1, AFF3, MS4A1, PAX5, BLK, FCRL1, CD79A, LINC00926 
	   COBLL1, OSBPL10, EBF1, RALGPS2, CD22, BLNK, BCL11A, SPIB, ADAM28, AP002075.1 
	   COL19A1, IGHD, CD79B, RUBCNL, WDFY4, DENND5B, TCF4, IGKC, RHEX, HLA-DQA1 
Negative:  IL32, CD247, TRAC, DPYD, IL7R, PDE3B, CAMK4, INPP4B, AOAH, THEMIS 
	   ANXA1, NEAT1, LEF1, FNDC3B, RORA, SLCO3A1, TXK, TRBC2, S100A4, TRBC1 
	   CTSW, ARHGAP26, GZMM, SRGN, PLCB1, ADGRE5, NCALD, LINC01934, SAMD3, SYNE2 
PC_ 3 
Positive:  GZMB, CLEC4C, LILRA4, PTPRS, LINC00996, LINC01374, CO

In [5]:
atac <- FindVariableFeatures(atac)
atac <- ScaleData(atac)
atac <- RunPCA(atac, npcs = 20)
atac <- RunUMAP(atac, dims = 1:20)
atac <- RunTSNE(atac, dims = 1:20)

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”


Loading required package: Signac

Warning message:
“The `slot` argument of `SetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”
Centering and scaling data matrix

PC_ 1 
Positive:  BCL11B, ZNF831, SCML4, LEF1, KLF12, ITK, IKZF3, CD247, LBH, ABLIM1 
	   FYN, SBK1, APBA2, RCAN3, SYNE2, CAMK4, GATA3, BACH2, PTCH1, RHOH 
	   CD96, IGSF9B, CD6, LCK, SYTL2, RASGRF2, MGAT4A, NCK2, SPOCK2, DOCK9 
Negative:  PLXDC2, RAB31, FPR1, LYN, RIN2, RBM47, AL121899.2, ARHGEF10L, FGD4, FMNL2 
	   GAS7, PPCDC, TREM1, DAPK1, SULF2, DMXL2, BASP1, SLC24A4, MS4A6E, FAM49A 
	   FLT1, FRY, EPB41L3, CACNB4, TNS3, LHFPL2, KLF4, ATP8B4, KCNIP1, HCK 
PC_ 2 
Positive:  BLK, PAX5, PLEKHG1, CHL1, KLHL14, NTNG1, UGT8, BACE2, CNR1, COL19A1 
	   EBF1, PXDNL, LAMC1, OSBPL10, ROR1, MYBL2, MS4A1, KCNH8, DENND5B, ST18 
	   COBLL1, MICAL3, ARH

## edgeR for rna

In [90]:
Reductions(atac)

[1] "lsi"  "umap"

In [91]:
out = file.path(OUTDIR, "atac", "edger_frag.csv")

In [92]:
# create 'fake' samples
set.seed(42)  # Set a reproducible seed
dat = atac
dat@meta.data[["sample"]] = paste0("atac", sample(1:50, dim(dat)[2], replace = TRUE))
dat@meta.data[["combined"]] = paste(dat@meta.data[["sample"]], dat@meta.data[["Celltype"]], sep = "_")

y <- Seurat2PB(dat, sample="sample", cluster="Celltype")
y <- normLibSizes(y)
head(y)

An object of class "DGEList"
$counts
        atac1_clusterB cells atac1_clusterCD4+ T cells
PLCXD1                     2                         7
GTPBP6                     9                        36
PPP2R3B                    8                        14
SHOX                       0                         0
CRLF2                      1                         2
CSF2RA                     3                         1
        atac1_clusterCD8+ T cells atac1_clusterMonocytes atac1_clusterNK cells
PLCXD1                          2                     14                     0
GTPBP6                          9                     32                     1
PPP2R3B                        12                     11                     1
SHOX                            0                      0                     0
CRLF2                           2                      2                     0
CSF2RA                          0                     20                     0
        atac1_clusterT cells atac10_clusterB cells atac10_clusterCD4+ T cells
PLCXD1                     6                     2                          7
GTPBP6                    18                     8                         40
PPP2R3B                    7                     6                         23
SHOX                       0                     0                          0
CRLF2                      0                     0                          4
CSF2RA                     1                     0                          0
        atac10_clusterCD8+ T cells atac10_clusterDendritic cells
PLCXD1                          13                             2
GTPBP6                          22                             5
PPP2R3B                         11                             2
SHOX                             0                             0
CRLF2                            1                             2
CSF2RA                           0                             7
        atac10_clusterMonocytes atac10_clusterNK cells atac10_clusterT cells
PLCXD1                       12                      5                     1
GTPBP6                       29                     10                    14
PPP2R3B                      20                      3                     9
SHOX                          0                      0                     0
CRLF2                         6                      0                     2
CSF2RA                       28                      0                     0
        atac11_clusterB cells atac11_clusterCD4+ T cells
PLCXD1                      3                          7
GTPBP6                      1                         15
PPP2R3B                     3                          8
SHOX                        0                          0
CRLF2                       0                          0
CSF2RA                      0                          0
        atac11_clusterCD8+ T cells atac11_clusterDendritic cells
PLCXD1                           3                             1
GTPBP6                          15                             5
PPP2R3B                          8                             7
SHOX                             0                             0
CRLF2                            0                             6
CSF2RA                           0                             6
        atac11_clusterMonocytes atac11_clusterNK cells atac11_clusterT cells
PLCXD1                        8                      1                     2
GTPBP6                       19                      5                    20
PPP2R3B                       8                      6                    11
SHOX                          0                      0                     0
CRLF2                         3                      0                     2
CSF2RA                        6                      0                     0
        atac12_clusterB cells atac12_clusterCD4+ T cells
PLCXD1                      6      

In [94]:
for (this_emb in c("lsi", "umap")) {
  print(this_emb)
  # prepare embedding
  embedding_df <- as.data.frame(Embeddings(dat, reduction = this_emb))

  # Create data frame for aggregation
  embedding_df$cell_type <- dat@meta.data[["combined"]]

  # Compute average embedding per cell type
  pseudo_embedding <- embedding_df %>%
    group_by(cell_type) %>%
    summarise(across(where(is.numeric), mean)) %>%
    as.data.frame()

  # Store row names as cell types
  rownames(pseudo_embedding) <- pseudo_embedding$cell_type
  pseudo_embedding$cell_type <- NULL
  #head(pseudo_embedding)
  print(" > Finished embeddings")

  design <- model.matrix(~ ., data = pseudo_embedding)

  y <- estimateDisp(y, design) # 10m with PCA
  print(" > Finished dispersion estimation")
  fit <- glmQLFit(y, design)
  print(" > Finished fitting full")

  # Test overall significance of the embedding covariates
  # or get deviance explained
  # To test how much deviance is explained by embeddings:

  # Fit null model (intercept only)
  design_null <- model.matrix(~ 1, data = pseudo_embedding)
  fit_null <- glmQLFit(y, design_null)
  print(" > Finished fitting NULL")

  # Compute deviance explained
  dev_full <- deviance(fit)
  dev_null <- deviance(fit_null)

  prop_dev_explained <- (dev_null - dev_full) / dev_null

  if (this_emb == "pca") {
    prop = data.frame(pca = prop_dev_explained)
    write.csv(prop, out, row.names = F)
  } else if (this_emb == "lsi") {
    prop = data.frame(lsi = prop_dev_explained)
    write.csv(prop, out, row.names = F)
  } else {
    prop = read.csv(out)
    prop[[paste0("pca_", this_emb)]] = prop_dev_explained
    write.csv(prop, out, row.names = F)
    print(" > Finished write deviance explained")
  }
}